<div style="text-align: left; font-family: 'charter'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [19]:
# PDF Downloader

import os  # for file and directory manipulation
import random  # to generate random numbers
import time  # to manage time and take breaks in the script
import requests  # to make HTTP requests to web servers
from selenium import webdriver  # for automating web browsers
from selenium.webdriver.common.by import By  # to locate elements on a webpage
from selenium.webdriver.support.ui import WebDriverWait  # to wait until certain conditions are met on a webpage.
from selenium.webdriver.support import expected_conditions as EC  # to define expected conditions
from selenium.common.exceptions import StaleElementReferenceException  # To handle exceptions related to elements on the webpage that are no longer available.


# Extracting Tables (and data cleaning)

import pdfplumber  # for extracting text and metadata from PDF files
import pandas as pd  # for data manipulation and analysis
import os  # for interacting with the operating system
import unicodedata  # for manipulating Unicode data
import re  # for regular expressions operations
from datetime import datetime  # for working with dates and times
import locale  # for locale-specific formatting of numbers, dates, and currencies


# SQL tables

import psycopg2  # for interacting with PostgreSQL databases
from sqlalchemy import create_engine, text  # for creating and executing SQL queries using SQLAlchemy

In [141]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import unidecode

In [21]:
# Obtener variables de entorno
user = os.environ.get('CIUP_SQL_USER')
password = os.environ.get('CIUP_SQL_PASS')
host = os.environ.get('CIUP_SQL_HOST')
port = 5432
database = 'gdp_revisions_datasets'

# Verificar si todas las variables de entorno están definidas
if not all([host, user, password]):
    raise ValueError("Faltan algunas variables de entorno (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

# Crear la cadena de conexión
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Crear el motor de SQLAlchemy
engine = create_engine(connection_string)

# Definir la consulta SQL para importar datos
sector = 'gdp'  # Define el sector según tus necesidades
query = f"SELECT * FROM {sector}_monthly_growth_rates"

# Importar los datos a un DataFrame de pandas
df = pd.read_sql(query, engine)

# Ahora 'df' contiene los datos importados desde la base de datos
df.head() # Muestra las primeras filas del DataFrame para verificar

,year,id_ns,date,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,...,mar_2023,abr_2023,may_2023,jun_2023,jul_2023,ago_2023,sep_2023,oct_2023,nov_2023,dic_2023
0,2013,01,2013-01-04,5.9,5.3,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,02,2013-01-11,5.9,5.3,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,03,2013-01-18,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,04,2013-01-25,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,05,2013-02-01,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
print(df['sep_2011'].dtype)

float64


# Cargando desde excel

In [184]:
import pandas as pd

# Substitua 'arquivo.csv' pelo caminho do seu arquivo CSV
path = r'C:\Users\Jason Cruz\OneDrive\Documentos\coding_training\old_dataset\raw_data\2012\ns-03-2012.csv'

# Carrega o arquivo CSV para um DataFrame
df = pd.read_csv(path, delimiter=';')

# Exibe as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
df.head(5)

,SECTORES ECONÓMICOS,2010,Unnamed: 2,Unnamed: 3,2011,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,ECONOMIC SECTORS
0,NaN,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
3,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
4,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock


# Data clean-up

### Auxiliares

In [24]:
def remove_rare_characters_first_row(texto):
    texto = re.sub(r'\s*-\s*', '-', texto)  # Remueve espacios alrededor de guiones
    texto = re.sub(r'[^a-zA-Z0-9\s-]', '', texto)  # Remueve caracteres raros excepto letras, dígitos y guiones
    return texto

def remove_rare_characters(texto):
    return re.sub(r'[^a-zA-Z\s]', '', texto)

def remove_tildes(texto):
    return ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))

### Común

In [25]:
# 0.
def drop_nan_rows(df):
    df = df.dropna(how='all')
    return df

# 1. 
def drop_nan_columns(df):
    return df.dropna(axis=1, how='all')

# 2.
def swap_first_second_row(df):
    temp = df.iloc[0, 0]
    df.iloc[0, 0] = df.iloc[1, 0]
    df.iloc[1, 0] = temp

    temp = df.iloc[0, -1]
    df.iloc[0, -1] = df.iloc[1, -1]
    df.iloc[1, -1] = temp
    return df

# 8. 
def reset_index(df):
    df.reset_index(drop=True, inplace=True)
    return df

# 5.
def remove_digit_slash(df):
    # Aplica la función de reemplazo a la primera columna y a las dos últimas columnas
    df.iloc[:, [0, -2, -1]] = df.iloc[:, [0, -2, -1]].apply(lambda x: x.str.replace(r'\d+/', '', regex=True))
    return df

# 9. AUX (ROBUSTO)

def separate_text_digits(df):
    for index, row in df.iterrows():
        if any(char.isdigit() for char in str(row.iloc[-2])) and any(char.isalpha() for char in str(row.iloc[-2])):
            if pd.isnull(row.iloc[-1]):
                df.loc[index, df.columns[-1]] = ''.join(filter(lambda x: x.isalpha() or x == ' ', str(row.iloc[-2])))
                df.loc[index, df.columns[-2]] = ''.join(filter(lambda x: not (x.isalpha() or x == ' '), str(row.iloc[-2])))
            
            # Check if comma or dot is used as decimal separator
            if ',' in str(row.iloc[-2]):
                split_values = str(row.iloc[-2]).split(',')
            elif '.' in str(row.iloc[-2]):
                split_values = str(row.iloc[-2]).split('.')
            else:
                # If neither comma nor dot found, assume no decimal part
                split_values = [str(row.iloc[-2]), '']
                
            cleaned_integer = ''.join(filter(lambda x: x.isdigit() or x == '-', split_values[0]))
            cleaned_decimal = ''.join(filter(lambda x: x.isdigit(), split_values[1]))
            if cleaned_decimal:
                # Use comma as decimal separator
                cleaned_numeric = cleaned_integer + ',' + cleaned_decimal
            else:
                cleaned_numeric = cleaned_integer
            df.loc[index, df.columns[-2]] = cleaned_numeric
    return df


# 4. 
def extract_years(df):
    year_columns = [col for col in df.columns if re.match(r'\b\d{4}\b', col)]
    #print("Años (4 dígitos) extraídos:")
    #print(year_columns)
    return year_columns

# 6. 
def first_row_columns(df):
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    return df

# 15.
def clean_columns_values(df):
    df.columns = df.columns.str.lower()
    # Only normalize string column names
    df.columns = [unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8') if isinstance(col, str) else col for col in df.columns]
    df.columns = df.columns.str.replace(' ', '_').str.replace('ano', 'year').str.replace('-', '_')
    
    text_columns = df.select_dtypes(include='object').columns
    for col in df.columns:
        df.loc[:, col] = df[col].apply(lambda x: remove_tildes(x) if isinstance(x, str) else x)
        df.loc[:, col] = df[col].apply(lambda x: str(x).replace(',', '.') if isinstance(x, (int, float, str)) else x)
    df.loc[:, 'sectores_economicos'] = df['sectores_economicos'].str.lower()
    df.loc[:, 'economic_sectors'] = df['economic_sectors'].str.lower()
    df.loc[:, 'sectores_economicos'] = df['sectores_economicos'].apply(remove_rare_characters)
    df.loc[:, 'economic_sectors'] = df['economic_sectors'].apply(remove_rare_characters)
    return df

# 16.
def convertir_float(df):
    excluded_columns = ['sectores_economicos', 'economic_sectors']
    columns_to_convert = [col for col in df.columns if col not in excluded_columns]
    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
    return df

# 15.
def relocate_last_column(df):
    last_column = df.pop(df.columns[-1])
    df.insert(1, last_column.name, last_column)
    return df

### Exclusiva Tabla 1

In [26]:
# ATIPIC LAST COLUMNS
def relocate_last_columns(df):
    if not pd.isna(df.iloc[1, -1]):
        # Create a new column with NaN
        new_column = 'col_' + ''.join(map(str, np.random.randint(1, 5, size=1)))
        df[new_column] = np.nan
        
        # Get 'ECONOMIC SECTORS' and relocate
        insert_value_1 = df.iloc[0, -2]
        # Convert the value to string before assignment
        insert_value_1 = str(insert_value_1)
        # Ensure the dtype of the last column is object (string) to accommodate string values
        df.iloc[:, -1] = df.iloc[:, -1].astype('object')
        df.iloc[0, -1] = insert_value_1
        
        # NaN first obs
        df.iloc[0,-2] = np.nan
    return df

# Extraer meses

def get_months_sublist_list(df, year_columns):
    first_row = df.iloc[0]
    # Initialize the list of sublists
    months_sublist_list = []

    # Initialize the current sublist
    months_sublist = []

    # Iterate over the elements of the first row
    for item in first_row:
        # Check if the item meets the requirements
        if len(str(item)) == 3:
            months_sublist.append(item)
        elif '-' in item or str(item) == 'year':
            months_sublist.append(item)
            months_sublist_list.append(months_sublist)
            months_sublist = []

    # Add the last sublist if it's not empty
    if months_sublist:
        months_sublist_list.append(months_sublist)

    new_elements = []

    # Check if year_columns is not empty
    if year_columns:
        for i, year in enumerate(year_columns):
            # Check if index i is valid for quarters_sublist_list
            if i < len(months_sublist_list):
                for element in months_sublist_list[i]:
                    new_elements.append(f"{year}_{element}")
                    
    two_first_elements = df.iloc[0][:2].tolist()

    # Ensure that the two_first_elements are added if they are not in new_elements
    for index in range(len(two_first_elements) - 1, -1, -1):
        if two_first_elements[index] not in new_elements:
            new_elements.insert(0, two_first_elements[index])

    # Ensure that the length of new_elements matches the number of columns in df
    while len(new_elements) < len(df.columns):
        new_elements.append(None)

    temp_df = pd.DataFrame([new_elements], columns=df.columns)
    df.iloc[0] = temp_df.iloc[0]

    return df


def find_year_column(df):
    # List to store the found years
    found_years = []

    # Iterating over the column names of the DataFrame
    for column in df.columns:
        # Checking if the column name is a year (4 digits)
        if column.isdigit() and len(column) == 4:
            found_years.append(column)

    # If more than one year is found, do nothing
    if len(found_years) > 1:
        pass
    # If exactly one year is found, implement additional code
    elif len(found_years) == 1:
        # Getting the name of the found year
        year_name = found_years[0]
        print("The name of the column representing the year is:", year_name)

        # Getting the first row of the DataFrame
        first_row = df.iloc[0]

        # Searching for the first column containing the word "year" or some hyphen-separated expression
        column_contains_year = first_row[first_row.astype(str).str.contains(r'\byear\b')]

        if not column_contains_year.empty:
            # Getting the name of the first column containing 'year' or some hyphen-separated expression in the first row
            column_contains_year_name = column_contains_year.index[0]
            print("The name of the first column containing 'year' or some hyphen-separated expression in the first row is:", column_contains_year_name)

            # Getting the indices of the columns
            column_contains_year_index = df.columns.get_loc(column_contains_year_name)
            year_name_index = df.columns.get_loc(year_name)
            print("The index of the column containing 'year' is:", column_contains_year_index)
            print("The index of the column representing the year is:", year_name_index)

            # Checking if the column representing the year is to the right or to the left of column_contains_year
            if column_contains_year_index < year_name_index:
                print("The year column is to the right of the column containing 'year'.")
                # Adding one to the year
                new_year = str(int(year_name) - 1)
                # Renaming the column containing 'year' with the new year
                df.rename(columns={column_contains_year_name: new_year}, inplace=True)
                print(f"The column containing 'year' is now named '{new_year}'.")
            elif column_contains_year_index > year_name_index:
                print("The year column is to the left of the column containing 'year'.")
                # Subtracting one from the year
                new_year = str(int(year_name) + 1)
                # Renaming the year column with the new year
                df.rename(columns={column_contains_year_name: new_year}, inplace=True)
                print(f"The column containing 'year' is now named '{new_year}'.")
            else:
                print("The year column is in the same position as the column containing 'year'.")
        else:
            print("No columns containing 'year' were found in the first row.")
    # If no year is found, print a message
    else:
        print("No years were found in the column names.")
    
    return df


#

def clean_first_row(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            if isinstance(df.at[0, col], str):
                df.at[0, col] = df.at[0, col].lower()  # Convertir a minúsculas solo si es un objeto
                df.at[0, col] = remove_tildes(df.at[0, col])
                df.at[0, col] = remove_rare_characters_first_row(df.at[0, col])
                # Reemplazar 'ano' por 'year'
                df.at[0, col] = df.at[0, col].replace('ano', 'year')

    return df


def intercambiar_valores(df):
    # Verificar si hay al menos dos columnas en el DataFrame
    if len(df.columns) < 2:
        print("El DataFrame tiene menos de dos columnas. No se pueden intercambiar valores.")
        return df

    # Verificar si hay valores NaN en la última columna
    if df.iloc[:, -1].isnull().any():
        # Obtener índice de filas con NaN en la última columna
        last_column_rows_nan = df[df.iloc[:, -1].isnull()].index

        # Iterar sobre las filas con NaN en la última columna
        for idx in last_column_rows_nan:
            # Verificar si el índice está dentro del rango de las columnas
            if -2 >= -len(df.columns):
                # Intercambiar los valores de la última columna y la penúltima columna
                df.iloc[idx, -1], df.iloc[idx, -2] = df.iloc[idx, -2], df.iloc[idx, -1]
            else:
                print(f"Índice fuera de rango para la fila {idx}. No se pueden intercambiar valores.")

    return df



def replace_var_perc_first_column(df):
    # Regular expression to search for "Var. %" or "Var.%"
    regex = re.compile(r'Var\. ?%')

    # Iterate over the rows of the dataframe
    for index, row in df.iterrows():
        # Convert the value in the first column to a string
        value = str(row.iloc[0])

        # Check if the value matches the regular expression
        if regex.search(value):
            # Replace only the characters that match the regular expression
            df.at[index, df.columns[0]] = regex.sub("variacion porcentual", value)
    
    return df


# 8.
number_moving_average = 'three' # Keep a space at the end

def replace_number_moving_average(df):
    for index, row in df.iterrows():
        # Buscar la expresión regular en la penúltima o última columna
        if pd.notnull(row.iloc[-1]) and re.search(r'(\d\s*-)', str(row.iloc[-1])):
            df.at[index, df.columns[-1]] = re.sub(r'(\d\s*-)', f'{number_moving_average}-', str(row.iloc[-1]))
        #elif pd.notnull(row.iloc[-2]) and re.search(r'(\d\s*-)', str(row.iloc[-2])):
        #   df.at[index, df.columns[-2]] = re.sub(r'(\d\s*-)', f'{number_moving_average}-', str(row.iloc[-2]))
    return df


# 7.
def replace_var_perc_last_columns(df):
    # Expresión regular para buscar "Var. %" o "Var.%"
    regex = re.compile(r'(Var\. ?%)(.*)')

    # Iterar sobre las filas del dataframe
    for index, row in df.iterrows():
        # Verificar si el valor en la penúltima columna es una cadena no nula
        if isinstance(row.iloc[-2], str) and regex.search(row.iloc[-2]):
            # Realizar el reemplazo al final del valor de la penúltima columna
            replaced_text = regex.sub(r'\2 percent change', row.iloc[-2])
            df.at[index, df.columns[-2]] = replaced_text.strip()
        
        # Verificar si el valor en la última columna es una cadena no nula
        if isinstance(row.iloc[-1], str) and regex.search(row.iloc[-1]):
            # Realizar el reemplazo al final del valor de la última columna
            replaced_text = regex.sub(r'\2 percent change', row.iloc[-1])
            df.at[index, df.columns[-1]] = replaced_text.strip()
    
    return df

# Función para buscar y reemplazar en la segunda fila del DataFrame
def replace_first_dot(df):
    second_row = df.iloc[1]  # Segunda fila del DataFrame
    
    # Verificar si al menos una observación cumple con el patrón
    if any(isinstance(cell, str) and re.match(r'^\w+\.\s?\w+', cell) for cell in second_row):
        for col in df.columns:
            if isinstance(second_row[col], str):  # Verificar si el valor es una cadena
                if re.match(r'^\w+\.\s?\w+', second_row[col]):  # Verificar si cumple con el patrón Xxx.Xxx o Xxx. Xxx.
                    df.at[1, col] = re.sub(r'(\w+)\.(\s?\w+)', r'\1-\2', second_row[col], count=1)  # Reemplazar solo el primer punto
    return df

def drop_rare_caracter_row(df):
    # Buscar el caracter solitario "}" en cada fila y obtener un booleano para cada fila
    rare_caracter_row = df.apply(lambda row: '}' in row.values, axis=1)
    
    # Filtrar el DataFrame para eliminar las filas con el caracter solitario "}"
    df = df[~rare_caracter_row]
    
    return df

def split_column_by_pattern(df):
    # Iteramos sobre las columnas del dataframe
    for col in df.columns:
        # Verificamos si la segunda fila de la columna contiene el patrón
        if re.match(r'^[A-Z][a-z]+\.?\s[A-Z][a-z]+\.?$', str(df.iloc[1][col])):
            # Realizamos el split de la columna usando como criterio el espacio
            split_values = df[col].str.split(expand=True)
            # Guardamos los primeros valores en la columna original
            df[col] = split_values[0]
            # Guardamos los segundos valores en una nueva columna con el sufijo "_split"
            new_col_name = col + '_split'
            df.insert(df.columns.get_loc(col) + 1, new_col_name, split_values[1])
    return df

$\Large{\color{blue}{ns\_2014\_07}}$

se: sectores económicos

In [27]:
def swap_nan_se(df):
    # Check if the first observation of the first column is NaN
    if pd.isna(df.iloc[0, 0]) and df.iloc[0, 1] == "SECTORES ECONÓMICOS":
        # Create a temporary copy of the values
        column_1_value = df.iloc[0, 1]
        # Swap values in the original row
        df.iloc[0, 0] = column_1_value
        df.iloc[0, 1] = np.nan
        # Drop the second column
        df = df.drop(df.columns[1], axis=1)
    return df

$\Large{\color{blue}{set \ by \ sep}}$

In [98]:
def replace_set_sep(df):
    # Get the column names of the DataFrame
    columns = df.columns
    
    # Iterate over the columns
    for column in columns:
        # Check if the column contains the expression 'set'
        if 'set' in column:
            # Replace 'set' with 'sep' in the column name
            new_column = column.replace('set', 'sep')
            # Rename the column in the DataFrame
            df.rename(columns={column: new_column}, inplace=True)
    
    return df

$\Large{\color{blue}{strip}}$

In [179]:
def spaces_se_es(df):
    # Aplicar strip a las columnas 'sectores_economicos' y 'economic_sectors'
    df['sectores_economicos'] = df['sectores_economicos'].str.strip()
    df['economic_sectors'] = df['economic_sectors'].str.strip()
    return df

$\Large{\color{blue}{Replace \ other \ services}}$

In [182]:
def replace_services(df):
    # Verificamos si se encuentran los valores 'otros servicios' y 'other services'
    if ('servicios' in df['sectores_economicos'].values) and ('services' in df['economic_sectors'].values):
        # Reemplazamos los valores
        df['sectores_economicos'].replace({'servicios': 'otros servicios'}, inplace=True)
        df['economic_sectors'].replace({'services': 'other services'}, inplace=True)
    return df

# Exclusivas para tablas entregadas por el BCRP

In [103]:
# 1.

In [156]:
def limpiar_nombres_columnas(df):
    # Eliminar tildes y convertir a minúsculas los nombres de las columnas
    df.columns = [unidecode.unidecode(col).lower() for col in df.columns]
    df.columns = df.columns.str.replace(' ', '')
    return df

In [157]:
# 2.

In [158]:
def ajustar_nombres_columnas(df):
    # Comprobar que la primera observación de la primera columna sea NaN
    if pd.isna(df.iloc[0, 0]) and pd.isna(df.iloc[0, -1]):
        # Verificar los nombres de las columnas
        if df.columns[0] == "sectoreseconomicos" and df.columns[-1] == "economicsectors":
            # Reemplazar NaN por los nombres de las columnas correspondientes
            df.iloc[0, 0] = "sectores economicos"
            df.iloc[0, -1] = "economic sectors"
    return df

In [185]:
# 3.

In [186]:
def redondear_valores(df, decimales=1):
    # Iterar sobre todas las columnas del DataFrame
    for col in df.columns:
        # Verificar si la columna es de tipo float
        if df[col].dtype == 'float64':
            # Redondear los valores de la columna al número de decimales especificado
            df[col] = df[col].round(decimales)
    return df

# Test

In [187]:
df = limpiar_nombres_columnas(df)
df = ajustar_nombres_columnas(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
3,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
4,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
9,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals


In [188]:
df = drop_rare_caracter_row(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
3,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
4,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
9,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals


In [189]:
df = drop_nan_rows(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
2,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
3,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
4,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
6,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
8,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
9,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals
10,Hidrocarburos,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817,Fuel
12,Manufactura,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436,Manufacturing
13,De recursos primarios,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643,Based on raw materials


In [190]:
df = drop_nan_columns(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
2,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
3,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
4,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
6,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
8,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
9,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals
10,Hidrocarburos,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817,Fuel
12,Manufactura,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436,Manufacturing
13,De recursos primarios,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643,Based on raw materials


In [191]:
df = reset_index(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
1,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
2,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
3,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
4,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
5,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
6,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals
7,Hidrocarburos,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817,Fuel
8,Manufactura,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436,Manufacturing
9,De recursos primarios,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643,Based on raw materials


In [192]:
df = remove_digit_slash(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
1,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
2,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
3,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
4,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
5,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
6,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals
7,Hidrocarburos,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817,Fuel
8,Manufactura,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436,Manufacturing
9,De recursos primarios,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643,Based on raw materials


In [193]:
df = replace_var_perc_first_column(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
1,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
2,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
3,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
4,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
5,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
6,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals
7,Hidrocarburos,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817,Fuel
8,Manufactura,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436,Manufacturing
9,De recursos primarios,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643,Based on raw materials


In [194]:
df = replace_var_perc_last_columns(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
1,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
2,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
3,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
4,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
5,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
6,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals
7,Hidrocarburos,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817,Fuel
8,Manufactura,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436,Manufacturing
9,De recursos primarios,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643,Based on raw materials


In [195]:
df = replace_number_moving_average(df)
df

,sectoreseconomicos,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15,economicsectors
0,sectores economicos,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.,economic sectors
1,Agropecuario,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321,Agriculture and Livestock
2,Agrícola,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981,Agriculture
3,Pecuario,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849,Livestock
4,Pesca,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453,Fishing
5,Minería e hidrocarburos,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607,Mining and fuel
6,Minería metálica,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457,Metals
7,Hidrocarburos,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817,Fuel
8,Manufactura,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436,Manufacturing
9,De recursos primarios,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643,Based on raw materials


In [196]:
df = relocate_last_column(df)
df

,sectoreseconomicos,economicsectors,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15
0,sectores economicos,economic sectors,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Set.,Oct.,Nov.,Ene.-Nov.
1,Agropecuario,Agriculture and Livestock,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321
2,Agrícola,Agriculture,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981
3,Pecuario,Livestock,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849
4,Pesca,Fishing,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453
5,Minería e hidrocarburos,Mining and fuel,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607
6,Minería metálica,Metals,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457
7,Hidrocarburos,Fuel,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817
8,Manufactura,Manufacturing,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436
9,De recursos primarios,Based on raw materials,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643


In [197]:
df = clean_first_row(df)
df

,sectoreseconomicos,economicsectors,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15
0,sectores economicos,economic sectors,nov,dic,year,ene,feb,mar,abr,may,jun,jul,ago,set,oct,nov,ene-nov
1,Agropecuario,Agriculture and Livestock,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321
2,Agrícola,Agriculture,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981
3,Pecuario,Livestock,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849
4,Pesca,Fishing,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453
5,Minería e hidrocarburos,Mining and fuel,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607
6,Minería metálica,Metals,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457
7,Hidrocarburos,Fuel,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817
8,Manufactura,Manufacturing,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436
9,De recursos primarios,Based on raw materials,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643


In [198]:
df = find_year_column(df)
df

,sectoreseconomicos,economicsectors,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15
0,sectores economicos,economic sectors,nov,dic,year,ene,feb,mar,abr,may,jun,jul,ago,set,oct,nov,ene-nov
1,Agropecuario,Agriculture and Livestock,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321
2,Agrícola,Agriculture,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981
3,Pecuario,Livestock,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849
4,Pesca,Fishing,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453
5,Minería e hidrocarburos,Mining and fuel,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607
6,Minería metálica,Metals,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457
7,Hidrocarburos,Fuel,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817
8,Manufactura,Manufacturing,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436
9,De recursos primarios,Based on raw materials,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643


In [199]:
year_columns = extract_years(df)
year_columns

['2010', '2011']

In [200]:
df = get_months_sublist_list(df, year_columns)
df

,sectoreseconomicos,economicsectors,2010,unnamed:2,unnamed:3,2011,unnamed:5,unnamed:6,unnamed:7,unnamed:8,unnamed:9,unnamed:10,unnamed:11,unnamed:12,unnamed:13,unnamed:14,unnamed:15
0,sectores economicos,economic sectors,2010_nov,2010_dic,2010_year,2011_ene,2011_feb,2011_mar,2011_abr,2011_may,2011_jun,2011_jul,2011_ago,2011_set,2011_oct,2011_nov,2011_ene-nov
1,Agropecuario,Agriculture and Livestock,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321
2,Agrícola,Agriculture,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981
3,Pecuario,Livestock,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849
4,Pesca,Fishing,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453
5,Minería e hidrocarburos,Mining and fuel,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607
6,Minería metálica,Metals,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457
7,Hidrocarburos,Fuel,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817
8,Manufactura,Manufacturing,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436
9,De recursos primarios,Based on raw materials,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643


In [201]:
df = first_row_columns(df)
df

,sectores economicos,economic sectors,2010_nov,2010_dic,2010_year,2011_ene,2011_feb,2011_mar,2011_abr,2011_may,2011_jun,2011_jul,2011_ago,2011_set,2011_oct,2011_nov,2011_ene-nov
1,Agropecuario,Agriculture and Livestock,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321
2,Agrícola,Agriculture,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981
3,Pecuario,Livestock,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849
4,Pesca,Fishing,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453
5,Minería e hidrocarburos,Mining and fuel,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607
6,Minería metálica,Metals,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457
7,Hidrocarburos,Fuel,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817
8,Manufactura,Manufacturing,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436
9,De recursos primarios,Based on raw materials,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643
10,No primaria,Non-primary,18.93247207,13.95987248,16.88936506,15.318,11.582,10.42,4.894,5.598,3.799,2.589,2.961,0.411,-0.824,-0.66,4.845371731


In [202]:
df = clean_columns_values(df)
df

,sectores_economicos,economic_sectors,2010_nov,2010_dic,2010_year,2011_ene,2011_feb,2011_mar,2011_abr,2011_may,2011_jun,2011_jul,2011_ago,2011_set,2011_oct,2011_nov,2011_ene_nov
1,agropecuario,agriculture and livestock,6.758289944,7.854083094,4.286827564,5.073320478,5.66303992,0.555226023,2.994131764,1.092354006,3.920237783,9.651790326,6.526110054,1.552485163,2.088563401,2.652158412,3.683887321
2,agricola,agriculture,5.519857659,9.003014667,4.176112876,2.009846689,3.416731053,-3.614204424,2.325479421,-0.752698213,1.957355682,12.2149653,9.279213875,0.820505358,0.002176966,0.824622991,2.39628981
3,pecuario,livestock,8.22452649,6.553299455,4.437536655,8.746806431,8.294074219,6.73700271,4.501741053,5.292663002,7.662908109,5.897703746,3.441686465,2.310622855,4.397153123,4.770432583,5.612185849
4,pesca,fishing,-33.86918639,-21.48477088,-16.44431964,18.01840578,10.72472759,7.887871433,106.7392845,1.412057549,-14.67366025,76.3749942,84.08931268,32.52483623,32.53,1.29,25.6045453
5,mineria e hidrocarburos,mining and fuel,-3.567981264,1.289579297,-0.148622392,0.010585859,-3.519361347,2.429301901,-3.919327988,2.760030901,-5.570214801,-1.137655722,3.071412047,0.673250453,-0.70710132,-1.023076778,-0.63918607
6,mineria metalica,metals,-10.75622049,-4.981242178,-4.908845891,-6.142046546,-9.612504082,-1.135480375,-10.58516448,-3.014997208,-9.544498704,-4.479623467,2.156216063,-0.932417345,-2.171322485,-1.672665536,-4.340141457
7,hidrocarburos,fuel,45.84460033,44.89061757,29.45930623,42.03004419,37.52117446,24.74905101,39.40662931,39.12087179,18.07618095,15.92961762,7.21301498,8.310906323,6.159783637,1.92380885,19.73764817
8,manufactura,manufacturing,14.72503218,9.54907546,13.61666709,15.09635772,11.412,10.834,9.648,5.603,2.509,3.938471933,5.298,1.764,-0.783,-0.73465747,5.63474436
9,de recursos primarios,based on raw materials,-5.767923266,-11.77852779,-2.338641499,13.635,10.204,13.035,37.258,5.465,-3.804809203,11.531,21.28,10.943,-0.612,-1.188,10.14216643
10,no primaria,nonprimary,18.93247207,13.95987248,16.88936506,15.318,11.582,10.42,4.894,5.598,3.799,2.589,2.961,0.411,-0.824,-0.66,4.845371731


In [203]:
df = convertir_float(df)
df

,sectores_economicos,economic_sectors,2010_nov,2010_dic,2010_year,2011_ene,2011_feb,2011_mar,2011_abr,2011_may,2011_jun,2011_jul,2011_ago,2011_set,2011_oct,2011_nov,2011_ene_nov
1,agropecuario,agriculture and livestock,6.758290,7.854083,4.286828,5.073320,5.663040,0.555226,2.994132,1.092354,3.920238,9.651790,6.526110,1.552485,2.088563,2.652158,3.683887
2,agricola,agriculture,5.519858,9.003015,4.176113,2.009847,3.416731,-3.614204,2.325479,-0.752698,1.957356,12.214965,9.279214,0.820505,0.002177,0.824623,2.396290
3,pecuario,livestock,8.224526,6.553299,4.437537,8.746806,8.294074,6.737003,4.501741,5.292663,7.662908,5.897704,3.441686,2.310623,4.397153,4.770433,5.612186
4,pesca,fishing,-33.869186,-21.484771,-16.444320,18.018406,10.724728,7.887871,106.739284,1.412058,-14.673660,76.374994,84.089313,32.524836,32.530000,1.290000,25.604545
5,mineria e hidrocarburos,mining and fuel,-3.567981,1.289579,-0.148622,0.010586,-3.519361,2.429302,-3.919328,2.760031,-5.570215,-1.137656,3.071412,0.673250,-0.707101,-1.023077,-0.639186
6,mineria metalica,metals,-10.756220,-4.981242,-4.908846,-6.142047,-9.612504,-1.135480,-10.585164,-3.014997,-9.544499,-4.479623,2.156216,-0.932417,-2.171322,-1.672666,-4.340141
7,hidrocarburos,fuel,45.844600,44.890618,29.459306,42.030044,37.521174,24.749051,39.406629,39.120872,18.076181,15.929618,7.213015,8.310906,6.159784,1.923809,19.737648
8,manufactura,manufacturing,14.725032,9.549075,13.616667,15.096358,11.412000,10.834000,9.648000,5.603000,2.509000,3.938472,5.298000,1.764000,-0.783000,-0.734657,5.634744
9,de recursos primarios,based on raw materials,-5.767923,-11.778528,-2.338641,13.635000,10.204000,13.035000,37.258000,5.465000,-3.804809,11.531000,21.280000,10.943000,-0.612000,-1.188000,10.142166
10,no primaria,nonprimary,18.932472,13.959872,16.889365,15.318000,11.582000,10.420000,4.894000,5.598000,3.799000,2.589000,2.961000,0.411000,-0.824000,-0.660000,4.845372


In [204]:
df = replace_set_sep(df)
df

,sectores_economicos,economic_sectors,2010_nov,2010_dic,2010_year,2011_ene,2011_feb,2011_mar,2011_abr,2011_may,2011_jun,2011_jul,2011_ago,2011_sep,2011_oct,2011_nov,2011_ene_nov
1,agropecuario,agriculture and livestock,6.758290,7.854083,4.286828,5.073320,5.663040,0.555226,2.994132,1.092354,3.920238,9.651790,6.526110,1.552485,2.088563,2.652158,3.683887
2,agricola,agriculture,5.519858,9.003015,4.176113,2.009847,3.416731,-3.614204,2.325479,-0.752698,1.957356,12.214965,9.279214,0.820505,0.002177,0.824623,2.396290
3,pecuario,livestock,8.224526,6.553299,4.437537,8.746806,8.294074,6.737003,4.501741,5.292663,7.662908,5.897704,3.441686,2.310623,4.397153,4.770433,5.612186
4,pesca,fishing,-33.869186,-21.484771,-16.444320,18.018406,10.724728,7.887871,106.739284,1.412058,-14.673660,76.374994,84.089313,32.524836,32.530000,1.290000,25.604545
5,mineria e hidrocarburos,mining and fuel,-3.567981,1.289579,-0.148622,0.010586,-3.519361,2.429302,-3.919328,2.760031,-5.570215,-1.137656,3.071412,0.673250,-0.707101,-1.023077,-0.639186
6,mineria metalica,metals,-10.756220,-4.981242,-4.908846,-6.142047,-9.612504,-1.135480,-10.585164,-3.014997,-9.544499,-4.479623,2.156216,-0.932417,-2.171322,-1.672666,-4.340141
7,hidrocarburos,fuel,45.844600,44.890618,29.459306,42.030044,37.521174,24.749051,39.406629,39.120872,18.076181,15.929618,7.213015,8.310906,6.159784,1.923809,19.737648
8,manufactura,manufacturing,14.725032,9.549075,13.616667,15.096358,11.412000,10.834000,9.648000,5.603000,2.509000,3.938472,5.298000,1.764000,-0.783000,-0.734657,5.634744
9,de recursos primarios,based on raw materials,-5.767923,-11.778528,-2.338641,13.635000,10.204000,13.035000,37.258000,5.465000,-3.804809,11.531000,21.280000,10.943000,-0.612000,-1.188000,10.142166
10,no primaria,nonprimary,18.932472,13.959872,16.889365,15.318000,11.582000,10.420000,4.894000,5.598000,3.799000,2.589000,2.961000,0.411000,-0.824000,-0.660000,4.845372


In [205]:
df = spaces_se_es(df)
df

,sectores_economicos,economic_sectors,2010_nov,2010_dic,2010_year,2011_ene,2011_feb,2011_mar,2011_abr,2011_may,2011_jun,2011_jul,2011_ago,2011_sep,2011_oct,2011_nov,2011_ene_nov
1,agropecuario,agriculture and livestock,6.758290,7.854083,4.286828,5.073320,5.663040,0.555226,2.994132,1.092354,3.920238,9.651790,6.526110,1.552485,2.088563,2.652158,3.683887
2,agricola,agriculture,5.519858,9.003015,4.176113,2.009847,3.416731,-3.614204,2.325479,-0.752698,1.957356,12.214965,9.279214,0.820505,0.002177,0.824623,2.396290
3,pecuario,livestock,8.224526,6.553299,4.437537,8.746806,8.294074,6.737003,4.501741,5.292663,7.662908,5.897704,3.441686,2.310623,4.397153,4.770433,5.612186
4,pesca,fishing,-33.869186,-21.484771,-16.444320,18.018406,10.724728,7.887871,106.739284,1.412058,-14.673660,76.374994,84.089313,32.524836,32.530000,1.290000,25.604545
5,mineria e hidrocarburos,mining and fuel,-3.567981,1.289579,-0.148622,0.010586,-3.519361,2.429302,-3.919328,2.760031,-5.570215,-1.137656,3.071412,0.673250,-0.707101,-1.023077,-0.639186
6,mineria metalica,metals,-10.756220,-4.981242,-4.908846,-6.142047,-9.612504,-1.135480,-10.585164,-3.014997,-9.544499,-4.479623,2.156216,-0.932417,-2.171322,-1.672666,-4.340141
7,hidrocarburos,fuel,45.844600,44.890618,29.459306,42.030044,37.521174,24.749051,39.406629,39.120872,18.076181,15.929618,7.213015,8.310906,6.159784,1.923809,19.737648
8,manufactura,manufacturing,14.725032,9.549075,13.616667,15.096358,11.412000,10.834000,9.648000,5.603000,2.509000,3.938472,5.298000,1.764000,-0.783000,-0.734657,5.634744
9,de recursos primarios,based on raw materials,-5.767923,-11.778528,-2.338641,13.635000,10.204000,13.035000,37.258000,5.465000,-3.804809,11.531000,21.280000,10.943000,-0.612000,-1.188000,10.142166
10,no primaria,nonprimary,18.932472,13.959872,16.889365,15.318000,11.582000,10.420000,4.894000,5.598000,3.799000,2.589000,2.961000,0.411000,-0.824000,-0.660000,4.845372


In [206]:
df = replace_services(df)
df

,sectores_economicos,economic_sectors,2010_nov,2010_dic,2010_year,2011_ene,2011_feb,2011_mar,2011_abr,2011_may,2011_jun,2011_jul,2011_ago,2011_sep,2011_oct,2011_nov,2011_ene_nov
1,agropecuario,agriculture and livestock,6.758290,7.854083,4.286828,5.073320,5.663040,0.555226,2.994132,1.092354,3.920238,9.651790,6.526110,1.552485,2.088563,2.652158,3.683887
2,agricola,agriculture,5.519858,9.003015,4.176113,2.009847,3.416731,-3.614204,2.325479,-0.752698,1.957356,12.214965,9.279214,0.820505,0.002177,0.824623,2.396290
3,pecuario,livestock,8.224526,6.553299,4.437537,8.746806,8.294074,6.737003,4.501741,5.292663,7.662908,5.897704,3.441686,2.310623,4.397153,4.770433,5.612186
4,pesca,fishing,-33.869186,-21.484771,-16.444320,18.018406,10.724728,7.887871,106.739284,1.412058,-14.673660,76.374994,84.089313,32.524836,32.530000,1.290000,25.604545
5,mineria e hidrocarburos,mining and fuel,-3.567981,1.289579,-0.148622,0.010586,-3.519361,2.429302,-3.919328,2.760031,-5.570215,-1.137656,3.071412,0.673250,-0.707101,-1.023077,-0.639186
6,mineria metalica,metals,-10.756220,-4.981242,-4.908846,-6.142047,-9.612504,-1.135480,-10.585164,-3.014997,-9.544499,-4.479623,2.156216,-0.932417,-2.171322,-1.672666,-4.340141
7,hidrocarburos,fuel,45.844600,44.890618,29.459306,42.030044,37.521174,24.749051,39.406629,39.120872,18.076181,15.929618,7.213015,8.310906,6.159784,1.923809,19.737648
8,manufactura,manufacturing,14.725032,9.549075,13.616667,15.096358,11.412000,10.834000,9.648000,5.603000,2.509000,3.938472,5.298000,1.764000,-0.783000,-0.734657,5.634744
9,de recursos primarios,based on raw materials,-5.767923,-11.778528,-2.338641,13.635000,10.204000,13.035000,37.258000,5.465000,-3.804809,11.531000,21.280000,10.943000,-0.612000,-1.188000,10.142166
10,no primaria,nonprimary,18.932472,13.959872,16.889365,15.318000,11.582000,10.420000,4.894000,5.598000,3.799000,2.589000,2.961000,0.411000,-0.824000,-0.660000,4.845372


In [207]:
df = redondear_valores(df, decimales=1)
df

,sectores_economicos,economic_sectors,2010_nov,2010_dic,2010_year,2011_ene,2011_feb,2011_mar,2011_abr,2011_may,2011_jun,2011_jul,2011_ago,2011_sep,2011_oct,2011_nov,2011_ene_nov
1,agropecuario,agriculture and livestock,6.8,7.9,4.3,5.1,5.7,0.6,3.0,1.1,3.9,9.7,6.5,1.6,2.1,2.7,3.7
2,agricola,agriculture,5.5,9.0,4.2,2.0,3.4,-3.6,2.3,-0.8,2.0,12.2,9.3,0.8,0.0,0.8,2.4
3,pecuario,livestock,8.2,6.6,4.4,8.7,8.3,6.7,4.5,5.3,7.7,5.9,3.4,2.3,4.4,4.8,5.6
4,pesca,fishing,-33.9,-21.5,-16.4,18.0,10.7,7.9,106.7,1.4,-14.7,76.4,84.1,32.5,32.5,1.3,25.6
5,mineria e hidrocarburos,mining and fuel,-3.6,1.3,-0.1,0.0,-3.5,2.4,-3.9,2.8,-5.6,-1.1,3.1,0.7,-0.7,-1.0,-0.6
6,mineria metalica,metals,-10.8,-5.0,-4.9,-6.1,-9.6,-1.1,-10.6,-3.0,-9.5,-4.5,2.2,-0.9,-2.2,-1.7,-4.3
7,hidrocarburos,fuel,45.8,44.9,29.5,42.0,37.5,24.7,39.4,39.1,18.1,15.9,7.2,8.3,6.2,1.9,19.7
8,manufactura,manufacturing,14.7,9.5,13.6,15.1,11.4,10.8,9.6,5.6,2.5,3.9,5.3,1.8,-0.8,-0.7,5.6
9,de recursos primarios,based on raw materials,-5.8,-11.8,-2.3,13.6,10.2,13.0,37.3,5.5,-3.8,11.5,21.3,10.9,-0.6,-1.2,10.1
10,no primaria,nonprimary,18.9,14.0,16.9,15.3,11.6,10.4,4.9,5.6,3.8,2.6,3.0,0.4,-0.8,-0.7,4.8
